In [2]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (256, 256))
        
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
        mask = cv2.resize(mask, (256, 256))
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

In [3]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = 'images_train\input'
train_mask_dir = 'images_train\output'
val_image_dir = 'images_val\input'
val_mask_dir = 'images_val\output'
test_image_dir = 'images_test\input'
test_mask_dir = 'images_test\output'

train_dataset = CoronarySmallDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [4]:
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

In [5]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, masks in tqdm(train_loader):
            images = images.to(device)
            masks = masks.to(device)

            # print(images.size())
            # print(images)
            # print(masks.size())
            # print(masks)

            optimizer.zero_grad()
            outputs = model(images)

            # print(outputs.size())
            # print(outputs)
            
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                
                val_loss += loss.item() * images.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'large_RGB_model.pth')


In [5]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

  0%|          | 0/70 [00:00<?, ?it/s]

100%|██████████| 70/70 [04:58<00:00,  4.27s/it]


Epoch 1/50, Train Loss: 0.5750, Val Loss: 0.5172


100%|██████████| 70/70 [05:03<00:00,  4.34s/it]


Epoch 2/50, Train Loss: 0.4530, Val Loss: 0.4443


...

Epoch 48/50, Train Loss: 0.0154, Val Loss: 0.0309


100%|██████████| 70/70 [04:54<00:00,  4.21s/it]


Epoch 49/50, Train Loss: 0.0147, Val Loss: 0.0300


100%|██████████| 70/70 [04:58<00:00,  4.26s/it]


Epoch 50/50, Train Loss: 0.0137, Val Loss: 0.0295


In [10]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms


model.load_state_dict(torch.load('large_RGB_model.pth'))
model.eval()

def show_image(type, image_name):
    dir = f"images_test\{type}"
    print(dir)
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    cv2.imshow(type, img)

def predict(image_name):
    dir = 'images_test\input'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = pred.squeeze().cpu().numpy()
        print(pred)
        # pred = (pred > 0.5).astype(np.uint8)
    R, G, B = pred[0], pred[1], pred[2]
    # cv2.imshow('R', R)
    # cv2.imshow('G', G)
    # cv2.imshow('B', B)
    pred_image = np.zeros((256, 256, 3), dtype=np.uint8)
    pred_image[..., 0] = (R * 255).astype(np.uint8)
    pred_image[..., 1] = (G * 255).astype(np.uint8)
    pred_image[..., 2] = (B * 255).astype(np.uint8)
    print(pred_image)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

In [11]:
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
# image_name = "13c2ur549vohc0jat2wd23me01_25.png"
# image_name = "131aedfhs6pnf1fvtvp49h4bhdjeabmt22_31.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
show_image("input", image_name)
show_image("output", image_name)
predict(image_name)

images_test\input
images_test\output


[[[0.00742865 0.00748531 0.0067906  ... 0.00790011 0.00866487 0.0054512 ]
  [0.00777635 0.00731615 0.00688989 ... 0.00528953 0.0062951  0.00730184]
  [0.00734305 0.0083128  0.00749891 ... 0.00669092 0.00789471 0.00755161]
  ...
  [0.0076694  0.00731048 0.00650615 ... 0.007401   0.00797816 0.00681169]
  [0.00904196 0.00750628 0.00746334 ... 0.00788658 0.00934628 0.00891352]
  [0.01533702 0.01126611 0.0091506  ... 0.00854262 0.00901388 0.00743356]]

 [[0.00518329 0.00575314 0.00538994 ... 0.00588013 0.00608126 0.00520962]
  [0.00489234 0.00643432 0.0056347  ... 0.00404436 0.0047843  0.00605669]
  [0.0050577  0.00678637 0.00608931 ... 0.00546005 0.00618416 0.00643466]
  ...
  [0.00530721 0.0055788  0.00506988 ... 0.00584189 0.00629676 0.0059546 ]
  [0.00589677 0.00580637 0.00577317 ... 0.00645069 0.00717541 0.00729802]
  [0.00977124 0.00804246 0.00637877 ... 0.00610758 0.00576341 0.00526816]]

 [[0.01235392 0.01217654 0.01135601 ... 0.00767267 0.00679398 0.00886894]
  [0.01039689 0.009249

In [11]:
model.load_state_dict(torch.load('large_RGB_model.pth'))

def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

model.eval()
test_loss = 0.0
dice_scores = []

with torch.no_grad():
    for images, masks in test_loader:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)
        
        test_loss += loss.item() * images.size(0)
        dice_scores.append(dice_coefficient(masks, outputs))

test_loss = test_loss / len(test_loader.dataset)
mean_dice = torch.mean(torch.tensor(dice_scores))

print(f'Test Loss: {test_loss:.4f}')
print(f'Mean Dice Coefficient: {mean_dice:.4f}')


Test Loss: 0.5773
Mean Dice Coefficient: 0.6904


In [ ]:
from skimage import morphology

def post_process(prediction):
    prediction_np = prediction.cpu().numpy()
    processed = morphology.remove_small_objects(prediction_np > 0.5, min_size=100)
    processed = morphology.remove_small_holes(processed, area_threshold=100)
    return torch.tensor(processed, device=device)

post_processed_predictions = [post_process(pred) for pred in outputs]


In [ ]:
from flask import Flask, request, jsonify
import io

app = Flask(__name__)
model.load_state_dict(torch.load('model.pth'))
model.eval()

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img_bytes = file.read()
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    img = cv2.resize(img, (512, 512))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = pred.squeeze().cpu().numpy()
        pred = (pred > 0.5).astype(np.uint8)
    
    _, buffer = cv2.imencode('.png', pred)
    response = io.BytesIO(buffer)
    response.seek(0)
    return response, 200, {'Content-Type': 'image/png'}

if __name__ == '__main__':
    app.run()
